compared with sg_2.ipynb


iccv-model 4 version 2: pocca + new trick + no merging + 100 epoch + 16 patch_num + new_fps

In [ ]:
import os
import torch

import sys
sys.path.append("..") 

from models.iccv_model_3 import SimAttention_ICCV_3
from data.shapenet_loader import ShapeNetCLS

# set gpu
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# hy-paras
BATCS_SIZE = 16
PATCH_NUM = 16
EPOCHS = 100

# data
root = r'/p/home/jusers/li39/juwels/H_AttentionProject'
dataset = ShapeNetCLS(root, 1024)
trainDataLoader = torch.utils.data.DataLoader(dataset, 
        batch_size=BATCS_SIZE, 
        shuffle=True, 
        num_workers=8, 
        pin_memory=True)

# set model
model = SimAttention_ICCV_3(patch_num=PATCH_NUM)
# optimizer = torch.optim.SGD(model.parameters(), lr=0.01, momentum=0.9)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3, weight_decay=1e-6)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=EPOCHS, eta_min=1e-5)

# scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=50, gamma=0.3)

def train_1_epoch(model, optimizer, data_loader, device):
        model.train()
        mean_loss = torch.zeros(1).to(device)
        optimizer.zero_grad()
        for step, data in enumerate(data_loader):
                aug1, aug2 = data
                loss = model(aug1.to(device), aug2.to(device))
                loss = loss.mean()
                loss.backward()
                mean_loss = (mean_loss * step + loss.detach()) / (step + 1)  # update mean losses
                optimizer.step()
                optimizer.zero_grad()
        return mean_loss.item()
    
print('start training!')
for epoch in range(0, EPOCHS):
    print('--------Epoch {} is running--------'.format(epoch))
    loss = train_1_epoch(model, optimizer, trainDataLoader, device)
    print("Loss: ", loss)
    print('\n')
    scheduler.step()
    if epoch%5==0 or (epoch+1)==EPOCHS:
        weight_name = 'cls_dg_iccv4_100_' + str(epoch) + '.pth'
        weight_path = os.path.join(r'/p/home/jusers/li39/juwels/H_AttentionProject/weights/iccv_4', weight_name)
        torch.save(model.state_dict(), weight_path)
        print('Model Saved!')

/p/project/hai_ml_gegpr/miniconda3/envs/my_env/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


start training!
--------Epoch 0 is running--------
Loss:  0.5005678534507751


Model Saved!
--------Epoch 1 is running--------
Loss:  0.43452188372612


--------Epoch 2 is running--------
Loss:  0.36793071031570435


--------Epoch 3 is running--------
Loss:  0.31868091225624084


--------Epoch 4 is running--------
Loss:  0.2967781126499176


--------Epoch 5 is running--------
Loss:  0.30268964171409607


Model Saved!
--------Epoch 6 is running--------
Loss:  0.2735309302806854


--------Epoch 7 is running--------
Loss:  0.26887020468711853


--------Epoch 8 is running--------
Loss:  0.2578388452529907


--------Epoch 9 is running--------
Loss:  0.32229864597320557


--------Epoch 10 is running--------
Loss:  0.2866976261138916


Model Saved!
--------Epoch 11 is running--------
Loss:  0.23517444729804993


--------Epoch 12 is running--------
Loss:  0.22300472855567932


--------Epoch 13 is running--------
Loss:  0.22262218594551086


--------Epoch 14 is running--------
Loss:  0.20373776

换成loginnode boster之后

In [2]:
import torch
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

/p/project/hai_ml_gegpr/miniconda3/envs/my_env/lib/python3.10/site-packages/torch/cuda/__init__.py:107: UserWarning: CUDA initialization: Unexpected error from cudaGetDeviceCount(). Did you run some cuda functions before calling NumCudaDevices() that might have already set an error? Error 805: MPS client failed to connect to the MPS control daemon or the MPS server (Triggered internally at /opt/conda/conda-bld/pytorch_1673856626795/work/c10/cuda/CUDAFunctions.cpp:109.)
  return torch._C._cuda_getDeviceCount() > 0


In [3]:
device

device(type='cpu')

果然也只有booster才有GPU，但是目前GPU的环境启动不起来，等吧。